In [1]:
import os
import cobra
if not 'HOME' in os.environ:
    os.environ['HOME'] = '.' # set your HOME path if not found (windows OS)
%run ms_ext_symcom.py

modelseedpy 0.3.3


In [2]:
cobra_config = cobra.Configuration()
cobra_config.solver = 'cplex'
print(cobra_config.solver)

<module 'optlang.cplex_interface' from '/opt/env/python3_10_modelseed/lib/python3.10/site-packages/optlang/cplex_interface.py'>


In [3]:
from cobrakbase.io.read_kbase_zip import load_kbase_zip_object
template = load_kbase_zip_object('./data/GramNegModelTemplateV5_DNR_v2.JSON.zip')
# 0.5 O2 [c] + 4 H+ [c] + 3 Cytochrome c2+ [c] --> H2O [c] + 2 H+ [e] + 3 Cytochrome c3+ [c]
template.reactions.rxn14415_c.lower_bound = 0
# NAD [c] + CoA [c] + Pyruvate [c] --> NADH [c] + CO2 [c] + Acetyl-CoA [c]
template.reactions.rxn00154_c.lower_bound = 0
# Phosphate [c] + D-fructose-6-phosphate [c] --> H2O [c] + Acetylphosphate [c] + D-Erythrose4-phosphate [c]
template.reactions.rxn00548_c.lower_bound = 0
# NAD [c] + Glycerol-3-phosphate [c] <-- NADH [c] + H+ [c] + Glycerone-phosphate [c]
template.reactions.rxn00611_c.upper_bound = 0
genome_rhoda = load_genome_from_annotation_file('./data/annotation_rhoda.tsv')
genome_acido = load_genome_from_annotation_file('./data/annotation_acido.tsv')

project = SynComStudy(template)
project.genome_acido = genome_acido
project.genome_rhoda = genome_rhoda

cobrakbase 0.3.1


In [4]:
model_acido, model_rhoda = project.build_isolates()

INFO:modelseedpy.core.msatpcorrection:max_gapfilling: 10, best_score: 0.0
INFO:modelseedpy.core.msmodelutl:Expansion time:empty:0.03477632500107575
INFO:modelseedpy.core.msmodelutl:Filtered count:0 out of 0
INFO:modelseedpy.core.msmodelutl:Expansion time:nitrate:0.03428320700004406
INFO:modelseedpy.core.msmodelutl:Filtered count:0 out of 0


gapfill reactions 33 new exchanges 4


INFO:modelseedpy.core.msatpcorrection:max_gapfilling: 10, best_score: 0.0
INFO:modelseedpy.core.msmodelutl:Expansion time:empty:0.033131992000562605
INFO:modelseedpy.core.msmodelutl:Filtered count:0 out of 0
INFO:modelseedpy.core.msmodelutl:Expansion time:nitrate:0.0325888789993769
INFO:modelseedpy.core.msmodelutl:Filtered count:0 out of 0


gapfill reactions 38 new exchanges 3
[R12] added L-Leucine [e0]
[R12] added ABC Transporter (L-Leucine) [c0]
[3H11] added LeuE
replace Flavodoxin with FAD/FADH2
rxn19704_c0 mass balance {'charge': -3, 'H': -3}
Fix Proton MassBalance [-3] [before] rxn19704_c0 NADP [c0] + H+ [c0] + FADH2 [c0] <=> NADPH [c0] + FAD [c0]
Fix Proton MassBalance [-3] [after ] rxn19704_c0 NADP [c0] + FADH2 [c0] <=> NADPH [c0] + FAD [c0] + 2 H+ [c0]
R12 Nitrate [c0] + Ubiquinol-8 [c0] --> H2O [c0] + Nitrite [c0] + Ubiquinone-8 [c0]
3H11 2 H+ [c0] + Nitrate [c0] + Ubiquinol-8 [c0] --> H2O [c0] + 2 H+ [e0] + Nitrite [c0] + Ubiquinone-8 [c0]


## Test isolate models

In [5]:
medium_base = {
    'EX_cpd00063_e0': 100.0, # Ca2+
    'EX_cpd00034_e0': 100.0, # Zn2+
    'EX_cpd00030_e0': 100.0, # Mn2+
    'EX_cpd00254_e0': 100.0, # Mg
    'EX_cpd10515_e0': 100.0, # Fe2+
    'EX_cpd00149_e0': 100.0, # Co2+
    'EX_cpd00099_e0': 100.0, # Cl-
    'EX_cpd11574_e0': 100.0, # Molybdate
    'EX_cpd00205_e0': 1.3415688, # K+
    'EX_cpd00971_e0': 34.9324073, # Na+
    'EX_cpd00058_e0': 100.0, # Cu2+
    'EX_cpd00009_e0': 100.0, # Phosphate
    'EX_cpd00048_e0': 100.0, # Sulfate
    'EX_cpd00001_e0': 100.0, # H2O
    'EX_cpd00067_e0': 100.0, # H+    
}

In [6]:
medium_acetate_nitrate =  {
    'EX_cpd00107_e0': 0.0, # leucine
    'EX_cpd00029_e0': 20.0, # Acetate
    'EX_cpd00209_e0': 10.0, # Nitrate
    'EX_cpd00013_e0': 100.0, # NH3
    'EX_cpd00305_e0': 100.0  # Thiamin
}
medium_acetate_nitrate.update(medium_base)
model_rhoda.medium = medium_acetate_nitrate
solution = model_rhoda.optimize()
model_rhoda.summary(solution)

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00009_e0,EX_cpd00009_e0,0.1543,0,0.00%
cpd00013_e0,EX_cpd00013_e0,1.868,0,0.00%
cpd00029_e0,EX_cpd00029_e0,10.81,2,99.98%
cpd00030_e0,EX_cpd00030_e0,0.0003735,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.0003735,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.04782,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.0003735,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.0003735,0,0.00%
cpd00067_e0,EX_cpd00067_e0,29.26,0,0.00%
cpd00099_e0,EX_cpd00099_e0,0.0003735,0,0.00%


In [7]:
medium_leucine_nitrate =  {
    'EX_cpd00107_e0': 4.0, # leucine
    'EX_cpd00029_e0': 0, # Acetate
    'EX_cpd00209_e0': 10.0, # Nitrate
    'EX_cpd00013_e0': 100.0, # NH3
    'EX_cpd00305_e0': 100.0  # Thiamin
}
medium_leucine_nitrate.update(medium_base)
model_rhoda.medium = medium_leucine_nitrate
solution = model_rhoda.optimize()
model_rhoda.summary(solution)

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00009_e0,EX_cpd00009_e0,0.1626,0,0.00%
cpd00030_e0,EX_cpd00030_e0,0.0003937,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.0003937,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.0504,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.0003937,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.0003937,0,0.00%
cpd00067_e0,EX_cpd00067_e0,18.42,0,0.00%
cpd00099_e0,EX_cpd00099_e0,0.0003937,0,0.00%
cpd00107_e0,EX_cpd00107_e0,4,6,99.98%
cpd00149_e0,EX_cpd00149_e0,0.0003937,0,0.00%


In [8]:
medium_acetate_nitrate =  {
    'EX_cpd00107_e0': 0.0, # leucine
    'EX_cpd00029_e0': 20.0, # Acetate
    'EX_cpd00209_e0': 10.0, # Nitrate
    'EX_cpd00013_e0': 100.0, # NH3
    'EX_cpd00305_e0': 100.0  # Thiamin
}
medium_acetate_nitrate.update(medium_base)
model_acido.medium = medium_acetate_nitrate
solution = model_acido.optimize()
model_acido.summary(solution)

Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00009_e0,EX_cpd00009_e0,0.1151,0,0.00%
cpd00029_e0,EX_cpd00029_e0,13.4,2,99.99%
cpd00030_e0,EX_cpd00030_e0,0.0002786,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.0002786,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.03595,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.0002786,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.0002786,0,0.00%
cpd00067_e0,EX_cpd00067_e0,32.25,0,0.00%
cpd00099_e0,EX_cpd00099_e0,0.0002786,0,0.00%
cpd00149_e0,EX_cpd00149_e0,0.0002786,0,0.00%


In [21]:
cobra.io.save_json_model(model_acido, './data/model_acido_gf2.json')
cobra.io.save_json_model(model_rhoda, './data/model_rhoda_gf2.json')

In [9]:
cf = CommFactory().with_model(model_acido, 0.7, 'A').with_model(model_rhoda, 0.3, 'R')
model_comm = _build(cf, with_gpr=False)
if 'bio1' not in model_comm.reactions:
    r_bio_sum = Reaction("bio1", "bio_com", "", 0, 1000)
    r_bio_sum.add_metabolites(
        {
            model_comm.metabolites.cpd11416_cA: -0.4,
            model_comm.metabolites.cpd11416_cR: -0.6,
        }
    )
    model_comm.add_reactions([r_bio_sum])
model_comm.objective = 'bio1'
model_comm

Name,model
Memory address,7fc4410b1c00
Number of metabolites,2401
Number of reactions,2596
Number of genes,0
Number of groups,0
Objective expression,1.0*bio1 - 1.0*bio1_reverse_b18f7
Compartments,"cA, e0, cR"


In [6]:
GSP_MEDIUM = {
    'EX_cpd00001_e0': 1000.0,
    'EX_cpd00013_e0': 1000.0,

    'EX_cpd00209_e0': 12.0,
    'EX_cpd00029_e0': 20.0, # acetate
 
    'EX_cpd00218_e0': 100.0,
    'EX_cpd00220_e0': 100.0,
    'EX_cpd00644_e0': 0.0002281,
    'EX_cpd00305_e0': 100.0,
    'EX_cpd00393_e0': 100.0,
    'EX_cpd03424_e0': 100.0,
    'EX_cpd00443_e0': 100.0,
    'EX_cpd00263_e0': 100.0,
    'EX_cpd00048_e0': 100.0,
    'EX_cpd00009_e0': 100.0,
    'EX_cpd00242_e0': 29.759425,
    'EX_cpd00205_e0': 1.3415688,
    'EX_cpd00063_e0': 100.0,
    'EX_cpd00971_e0': 34.9324073,
    'EX_cpd00099_e0': 100.0,
    'EX_cpd00254_e0': 100.0,
    'EX_cpd00030_e0': 100.0,
    'EX_cpd00058_e0': 100.0,
    'EX_cpd00034_e0': 100.0,
    'EX_cpd10515_e0': 100.0,
    'EX_cpd00149_e0': 100.0,
    'EX_cpd00244_e0': 100.0,
    'EX_cpd11574_e0': 100.0,
    'EX_cpd15574_e0': 100.0,
    'EX_cpd00067_e0': 100.0,
 }
model_comm.medium = {rxn_id: v for rxn_id, v in GSP_MEDIUM.items() if rxn_id in model_comm.reactions}

In [7]:
model_comm.reactions.ATPM_cA.lower_bound = 0
model_comm.reactions.ATPM_cR.lower_bound = 0

In [8]:
#model_comm.reactions.LeuE_cA.upper_bound = -10
solution_comm_wt = cobra.flux_analysis.pfba(model_comm)
model_comm.summary(solution_comm_wt)

/opt/env/python3_10_modelseed/lib/python3.10/site-packages/optlang/gurobi_interface.py:728: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self.problem.addConstr(lhs, sense, rhs, name=constraint.name)


Metabolite,Reaction,Flux,C-Number,C-Flux
cpd00009_e0,EX_cpd00009_e0,0.1719,0,0.00%
cpd00013_e0,EX_cpd00013_e0,2.081,0,0.00%
cpd00029_e0,EX_cpd00029_e0,12.57,2,99.98%
cpd00030_e0,EX_cpd00030_e0,0.0004162,0,0.00%
cpd00034_e0,EX_cpd00034_e0,0.0004162,0,0.00%
cpd00048_e0,EX_cpd00048_e0,0.05345,0,0.00%
cpd00058_e0,EX_cpd00058_e0,0.0004162,0,0.00%
cpd00063_e0,EX_cpd00063_e0,0.0004162,0,0.00%
cpd00067_e0,EX_cpd00067_e0,22.86,0,0.00%
cpd00099_e0,EX_cpd00099_e0,0.0004162,0,0.00%


In [9]:
model_comm.objective = 'ATPM_cR'
sol = cobra.flux_analysis.pfba(model_comm)
sol.fluxes['ATPM_cA'], sol.fluxes['ATPM_cR']

/opt/env/python3_10_modelseed/lib/python3.10/site-packages/optlang/gurobi_interface.py:728: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self.problem.addConstr(lhs, sense, rhs, name=constraint.name)


(0.0, 25.687500000000014)

In [28]:
sol.fluxes.to_csv('solution_ATPM_cR_no_acido_nar.csv')

In [30]:
model_comm.reactions.rxn09004_cA.lower_bound = -1000
model_comm.reactions.rxn09004_cA.upper_bound = 1000

In [38]:
model_comm.reactions.rxn00288_cA

Reaction identifier,rxn00288_cA
Name,Succinate:(acceptor) oxidoreductase [A]
Memory address,0x7f88193e0f10
Stoichiometry,cpd00015_cA + cpd00036_cA + cpd00067_cA --> cpd00106_cA + cpd00982_cA FAD [A] + Succinate [A] + H+ [A] --> Fumarate [A] + FADH2 [A]
GPR,
Lower bound,0
Upper bound,0


In [15]:
cobra.io.save_json_model(model_comm, './data/model_syncom.json')

In [22]:
model_comm.reactions.rxn00611_cA.upper_bound = 0
model_comm.reactions.rxn00611_cA

Reaction identifier,rxn00611_cA
Name,sn-Glycerol-3-phosphate:NAD+ 2-oxidoreductase [A]
Memory address,0x7f8819475e40
Stoichiometry,cpd00003_cA + cpd00080_cA <-- cpd00004_cA + cpd00067_cA + cpd00095_cA NAD [A] + Glycerol-3-phosphate [A] <-- NADH [A] + H+ [A] + Glycerone-phosphate [A]
GPR,
Lower bound,-1000
Upper bound,0


In [26]:
model_comm.reactions.rxn00288_cA.upper_bound = 0
model_comm.reactions.rxn00288_cR.upper_bound = 0

In [39]:
model_comm.reactions.rxn00288_cR

Reaction identifier,rxn00288_cR
Name,Succinate:(acceptor) oxidoreductase [R]
Memory address,0x7f88196dbc10
Stoichiometry,cpd00015_cR + cpd00036_cR + cpd00067_cR --> cpd00106_cR + cpd00982_cR FAD [R] + Succinate [R] + H+ [R] --> Fumarate [R] + FADH2 [R]
GPR,
Lower bound,0
Upper bound,0


In [32]:
for r in model_comm.reactions:
    v = sol.fluxes[r.id]
    if v!= 0:
        print(r.id, r.build_reaction_string(True), v)

rxn05561_cA H+ [e0] + Fumarate [e0] <=> H+ [A] + Fumarate [A] -5.560975609756097
rxn05319_cA H2O [e0] <=> H2O [A] -11.121951219512193
rxn00097_cA ATP [A] + AMP [A] + H+ [A] <=> 2 ADP [A] 10.243902439024389
rxn00336_cA Isocitrate [A] --> Succinate [A] + Glyoxalate [A] 5.560975609756097
rxn00248_cA NAD [A] + L-Malate [A] <=> NADH [A] + Oxaloacetate [A] + H+ [A] 5.560975609756097
rxn09176_cA ATP [A] + Phosphate [A] <=> ADP [A] + PPi [A] -10.243902439024389
rxn10122_cA NADH [A] + 4.5 H+ [A] + Ubiquinone-8 [A] --> NAD [A] + 3.5 H+ [e0] + Ubiquinol-8 [A] 6.439024390243903
rxn00175_cA ATP [A] + CoA [A] + Acetate [A] --> PPi [A] + AMP [A] + Acetyl-CoA [A] + H+ [A] 10.243902439024389
rxn09272_cA Succinate [A] + Ubiquinone-8 [A] --> Fumarate [A] + Ubiquinol-8 [A] 5.560975609756097
rxn01388_cA Isocitrate [A] <=> H2O [A] + cis-Aconitate [A] -5.560975609756097
rxn00974_cA Citrate [A] <=> H2O [A] + cis-Aconitate [A] 5.560975609756097
rxn00256_cA CoA [A] + H+ [A] + Citrate [A] <-- H2O [A] + Acetyl-Co